In [1]:
data_dir = '../../input/data/train_dataset/'
file_name = 'train_data.csv'
test_file_name = 'test_data.csv'

In [2]:
y_id = 'cycle' # 'user' or 'cycle'
y_method = 'last' # 'last' or 'next'
n_folds = 5

In [4]:
import sys
sys.path.append("..")

In [5]:
import os
import easydict
import time
from datetime import datetime

from fe.feature import FEPipeline
from fe.agg import *

In [6]:
def convert_time(s):
    timestamp = time.mktime(datetime.strptime(s, '%Y-%m-%d %H:%M:%S').timetuple())
    return int(timestamp)

In [7]:
def preprocess(df):
    preprocess_df_path = '/opt/ml/features/preprocess.csv'
    
    if not os.path.exists(preprocess_df_path):
        # YYYY-MM-DD HH:MM:SS -> sec format
        df['Timestamp'] = df['Timestamp'].apply(convert_time)
        
        # 한 문제 푸는데 걸린 시간
        df['time_diff'] = df['Timestamp'] - df['Timestamp'].shift(1)
        
        # userID 별 푼 문항의 누적 합
        df['UserCumtestnum'] = df.groupby(['userID'])['answerCode'].cumcount()

        # userID, KnowledgeTag 별 푼 문항의 누적 합
        df['UserTagCumtestnum'] = df.groupby(['userID', 'KnowledgeTag'])['answerCode'].cumcount()

        # userID, testId 별 푼 문항의 누적 합
        df['UserTestCumtestnum'] = df.groupby(['userID','testId'])['answerCode'].cumcount()

        testId2maxlen = df[['assessmentItemID', 'testId']].drop_duplicates().groupby('testId').size()
        # test의 문항 수
        df['TestSize'] = df.testId.map(testId2maxlen)
        # user가 같은 test를 여러 번 푼 것인지 나타낸 변수 (처음 품 : 0, 두번 품 : 1, 세번 품 : 2)
        df['Retest'] = df['UserTestCumtestnum'] // df['TestSize']

        # Cycle 별 푼 문항의 누적 합
        df['UserCycleCumtestnum'] = df['UserTestCumtestnum'] % df['TestSize']        

        # df.to_csv(preprocess_df_path, index=False)
        print('* Success to save preprocessed df')

    else:
        df = pd.read_csv(preprocess_df_path)
    
        print('* Success to load preprocessed df')
    
    return df

In [24]:
def set_y(df, y_id, y_method):
    if y_method == 'last':
        # y_id 별 마지막 행을 y로
        y = df.groupby(y_id)['answerCode'].apply(lambda x: list(x)[-1])
        y.name = 'y'
        df = df.merge(y, how="inner", on=y_id)

        # y_id 별 마지막 행 제거
        last_idx = df.groupby(y_id).apply(lambda x: x.index[-1]).values
        df = df.drop(last_idx)
    
    elif y_method == 'next':
        # 다음 answerCode를 y로
        df['y'] = df.answerCode.shift(-1)

        # y_id 별 마지막 행 제거
        last_idx = df.groupby(y_id).apply(lambda x: x.index[-1]).values
        df = df.drop(last_idx)
    
    df = df.reset_index(drop=True)
    
    print(f"* Success to set y by method '{y_method}'")
    
    return df

def make_cumdata(df):
    preprocess_df_path = '/opt/ml/features/cumdata.csv'
    
    if not os.path.exists(preprocess_df_path):
        temp_df = pd.DataFrame(df.userID)
        
        # test의 문항 번호
        temp_df['testNumber'] = [int(assessment[-3:]) for assessment in df.assessmentItemID]

        # userID 별 맞춘 문항의 누적 합
        temp_df["UserCumcorrectnum"] = df.groupby(['userID'])['answerCode'].apply(lambda x: x.cumsum().shift(1))
        # userID 별 누적 정답률
        temp_df["UserCumcorrectper"] = temp_df['UserCumcorrectnum'] / df['UserCumtestnum']

        # userID, KnowledgeTag 별 맞춘 문항의 누적 합
        temp_df["UserTagCumcorrectnum"] = df.groupby(['userID', 'KnowledgeTag'])['answerCode'].apply(lambda x: x.cumsum().shift(1))
        temp_df["UserTagCumcorrectnum"] = temp_df["UserTagCumcorrectnum"].fillna(0)
        # userID, KnowledgeTag 별 누적 정답률
        temp_df["UserTagCumcorrectper"] = temp_df['UserTagCumcorrectnum'] / df['UserTagCumtestnum']
        temp_df["UserTagCumcorrectper"] = temp_df["UserTagCumcorrectper"].fillna(0)

        # userID, testId 별 맞춘 문항의 누적 합
        temp_df["UserTestCumcorrectnum"] = df.groupby(['userID','testId'])['answerCode'].apply(lambda x: x.cumsum().shift(1))
        # userID, testId 별 누적 정답률
        temp_df["UserTestCumcorrectper"] = temp_df['UserTestCumcorrectnum'] / df['UserTestCumtestnum']

        # Cycle 별 맞춘 문항의 누적 합
        temp_df['UserCycleCumcorrectnum'] = df.groupby(['userID','testId','Retest'])['answerCode'].apply(lambda x: x.cumsum().shift(1))
        # Cycle 별 누적 정답률
        temp_df['UserCycleCumcorrectper'] = temp_df['UserCycleCumcorrectnum']/df['UserCycleCumtestnum']
        
        temp_df = temp_df.drop('userID', axis=1)
        # temp_df.to_csv(preprocess_df_path, index=False)
        
        print('* Success to save cumulative data')
    
    else:
        temp_df = pd.read_csv(preprocess_df_path)
    
        print('* Success to load cumulative data')

    df = pd.concat([df, temp_df], axis=1)
    
    return df

def y_id2columns(y_id):
    if y_id == 'user':
        y_id = 'userID'
    elif y_id == 'cycle':
        y_id = ['userID', 'testId', 'Retest']
    
    return y_id

In [8]:
t_df = pd.read_csv(os.path.join(data_dir, file_name))

In [9]:
tp_df = preprocess(t_df)

* Success to save preprocessed df


In [11]:
tp_df.head(3)

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,time_diff,UserCumtestnum,UserTagCumtestnum,UserTestCumtestnum,TestSize,Retest,UserCycleCumtestnum
0,0,A060001001,A060000001,1,1584976631,7224,NaN,0,0,0,6,0,0
1,0,A060001002,A060000001,1,1584976634,7225,3.0,1,0,1,6,0,1
2,0,A060001003,A060000001,1,1584976642,7225,8.0,2,1,2,6,0,2


In [17]:
len(t_df), len(tp_df)

(2266586, 2266586)

In [14]:
y_id = y_id2columns(y_id)

In [15]:
y_id

['userID', 'testId', 'Retest']

In [20]:
t_df = t_df.sort_values(by=['userID','Timestamp']).reset_index(drop=True)
len(t_df)

2266586

In [22]:
tpp_df = set_y(tp_df, y_id, y_method)

* Success to set y by method 'last'


In [23]:
tpp_df.head(3)

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,time_diff,UserCumtestnum,UserTagCumtestnum,UserTestCumtestnum,TestSize,Retest,UserCycleCumtestnum,y
0,0,A060001001,A060000001,1,1584976631,7224,NaN,0,0,0,6,0,0,1
1,0,A060001002,A060000001,1,1584976634,7225,3.0,1,0,1,6,0,1,1
2,0,A060001003,A060000001,1,1584976642,7225,8.0,2,1,2,6,0,2,1


In [27]:
len(t_df), len(tp_df), len(tpp_df)

(2266586, 2266586, 1893983)

In [28]:
tp_df.groupby(y_id)['answerCode'].apply(lambda x: list(x)[-1])

userID  testId      Retest
0       A020000172  0         0
        A020000174  0         0
        A020000181  0         0
        A020000182  0         1
        A020000183  0         1
                             ..
7440    A030000136  0         0
        A030000197  0         0
        A050000096  0         0
7441    A030000071  0         0
        A040000165  0         1
Name: answerCode, Length: 372603, dtype: int64

In [79]:
t_df['time_diff'].fillna(0, inplace=True)

In [80]:
t_df['time_diff']

0                0.0
1                3.0
2                8.0
3                7.0
4                7.0
             ...    
2266581         24.0
2266582    6632178.0
2266583         11.0
2266584         46.0
2266585         73.0
Name: time_diff, Length: 2266586, dtype: float64